### Пример проекта по визуализации геоданных

Библиотеки, которые нам понадобились:

**pandas** для сбора и анализа данных
**geocoder** - для получения коорднат из названия стран
**folium** - для нанесения данных на карту

Установка:
в пустой ячейке введите:
!pip install geocoder folium


https://proglib.io/p/python-web-map/
пошаговая инструкция по фолиуму

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

r = requests.get('https://www.worldometers.info/coronavirus/')


In [5]:
df = pd.read_html('https://www.worldometers.info/coronavirus/')[0]

In [6]:
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop
0,China,80849,+25,3199.0,10.0,66930.0,10720,3226.0,56.2
1,Italy,21157,NaN,1441.0,NaN,1966.0,17750,1518.0,349.9
2,Iran,13938,"+1,209",724.0,113.0,4339.0,8875,NaN,165.9
3,S. Korea,8162,+76,75.0,3.0,834.0,7253,59.0,159.2
4,Spain,6821,+430,208.0,12.0,517.0,6096,272.0,145.9
...,...,...,...,...,...,...,...,...,...
152,Eswatini,1,NaN,NaN,NaN,NaN,1,NaN,0.9
153,Togo,1,NaN,NaN,NaN,NaN,1,NaN,0.1
154,U.S. Virgin Islands,1,NaN,NaN,NaN,NaN,1,NaN,NaN
155,Uzbekistan,1,+1,NaN,NaN,NaN,1,NaN,NaN


In [32]:
# import geocoder
# lat = []
# lng = []
# for i in range(len(pickled)):
#     g = geocoder.arcgis(pickled['Country,Other'].iloc[i])
#     lat.append(g.json['lat'])
#     lng.append(g.json['lng'])

# pickled['lat'] = lat
# pickled['lng'] = lng

In [1]:
import geocoder
g = geocoder.arcgis('Russia')
g.json

{'address': 'Russia',
 'bbox': {'northeast': [84.9, 148.57137153400006],
  'southwest': [10.223526183000068, 45.039371534000075]},
 'confidence': 1,
 'lat': 61.989526183000066,
 'lng': 96.80537153400007,
 'ok': True,
 'quality': 'Locality',
 'raw': {'name': 'Russia',
  'extent': {'xmin': 45.039371534000075,
   'ymin': 10.223526183000068,
   'xmax': 148.57137153400006,
   'ymax': 84.9},
  'feature': {'geometry': {'x': 96.80537153400007, 'y': 61.989526183000066},
   'attributes': {'Score': 100, 'Addr_Type': 'Locality'}}},
 'score': 100,
 'status': 'OK'}

In [22]:
pickled = pd.read_csv("https://raw.githubusercontent.com/alnagaev/python_lessons/master/latlng.csv")
df['lat'] = pickled['lat']
df['lng'] = pickled['lng'] 

In [23]:
df['ratio'] = round(df['TotalDeaths']/df['TotalCases']*100, 2)
df[['Country,Other', 'ratio', ]].dropna().head()

 

,"Country,Other",ratio
0,China,3.96
1,Italy,6.81
2,Iran,5.19
3,S. Korea,0.92
4,Spain,3.05


In [24]:
from math import ceil
def get_radius(rat):
    if rat > 10:
        return 10
    elif rat < 1:
        return 1
    else:
        return ceil(rat)

In [25]:
dropped = df[['Country,Other', 'ratio', 'lat', 'lng', 'TotalCases']].dropna()
dropped['percent'] = round(dropped['TotalCases']/dropped['TotalCases'].sum(), 2)*100
dropped['percent'].head()

0    52.0
1    13.0
2     9.0
3     5.0
4     4.0
Name: percent, dtype: float64

In [26]:
import folium
fol_map = folium.Map(location=[36.567348, 103.930027], zoom_start=2)
for lat, lon, percent, total, country in zip(
                            dropped['lat'].tolist(), 
                            dropped['lng'].tolist(), 
                            dropped['percent'].tolist(),
                            dropped['TotalCases'].tolist(),
                            dropped['Country,Other'].tolist()
                                 ):
    folium.CircleMarker(location=[lat, lon], radius = get_radius(percent), popup= str(country)+' ' + str(total)+" total cases").add_to(fol_map)

In [19]:
fol_map